In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from math import isclose
from ase.formula import Formula

In [2]:
from matkit.pathway import get_balanced_count, get_relative_energy

### VASP Parameters

In [3]:
xc = 'RPBE' # 'PW91', 'PBE', 'RPBE'
encut = 400 # 400, 520
ispin = 2 # 1, 2
ivdw = 0 # 0, 10, 11, 12
concentration = 0 # 0.0, 0.1, 1.0

### CO<sub>2</sub>RR Correction
OCO backbone  
<https://doi.org/10.1038/s41467-018-03712-z>

In [4]:
data = pd.read_csv('data/gas_phase.csv', index_col=0)
data = data[(data.xc==xc)&(data.encut==encut)&(data.ispin==ispin)&(data.ivdw==ivdw)&(data.concentration==concentration)]
data = data.set_index('formula')

In [5]:
def get_chemical_equation(species): # 使用H2和H2O配平化学式中的H和O
    count = {'H': 0, 'O': 0}
    for formula, n in species.items():
        formula_count = Formula(formula).count()
        count['H'] += n*formula_count.get('H', 0)
        count['O'] += n*formula_count.get('O', 0)
    balanced_count = get_balanced_count(count, pe=False)
    chemical_equation = species.copy()
    chemical_equation.update({'H2O': balanced_count['H2O'], 'H2': balanced_count['H2']})
    return chemical_equation

In [6]:
reaction = pd.DataFrame(map(get_chemical_equation, [
    {'CO2': -1., 'CO': 1.},
    {'CO2': -1., 'CH4': 1.}, {'CO': -1., 'CH4': 1.},
    {'CO2': -1., 'HCOOH': 1.,}, {'CO': -1., 'HCOOH': 1.,},
    {'CO2': -1., 'CH3OH': 1.}, {'CO': -1., 'CH3OH': 1.,},
    {'CO2': -1., 'C2H4': 0.5}, {'CO': -1., 'C2H4': 0.5},
    {'CO2': -1., 'C2H6': 0.5}, {'CO': -1., 'C2H6': 0.5},
])).fillna(0) # 气相热化学反应

In [7]:
stoichiometry = reaction.assign(
    enthalpy_experimental=(reaction*data['enthalpy_experimental']).sum(axis=1),
    enthalpy_calculated=(reaction*data['enthalpy_calculated']).sum(axis=1),
) # 计算反应热(焓变)
stoichiometry = stoichiometry.assign(OCO_backbone=stoichiometry['CO2']+stoichiometry['HCOOH']) # 记录OCO骨架

In [8]:
model = LinearRegression() # 线性回归模型
X = stoichiometry[['OCO_backbone', 'CO']]
Y = stoichiometry.enthalpy_experimental - stoichiometry.enthalpy_calculated
model.fit(X, Y)
correction = dict(zip(['OCO_backbone', 'CO'], model.coef_))
correction # 校正值

{'OCO_backbone': 0.27386106522601406, 'CO': -0.08006323147346939}

In [9]:
data.loc[['CO2', 'HCOOH'], 'correction'] = correction['OCO_backbone']
data.loc['CO', 'correction'] = correction['CO']
data.correction = data.correction.fillna(0.)
free_energy = data.free_energy + data.correction
data = data.rename(columns={'free_energy': 'free_energy_uncorrected'}).assign(free_energy=free_energy)

#### 校正过程
Analysis of reaction enthalpies (in eV) of gas-phase thermochemical reactions.
ΔH_exp was the experimental value taken from Computational Chemistry Comparison and Benchmark DataBase (https://cccbdb.nist.gov/exp1x.asp),
ΔH_calc was RPBE calculated reaction enthalpies,
ΔH_corr was corrected reaction enthalpies that included -0.080 eV for CO and +0.274 eV for the OCO backbone.

In [10]:
def stoichiometric(n): # 格式化化学反应系数
    if isclose(n%1., 0.): # 整数判断
        coefficient = '{:d} '.format(int(n)).replace('1 ', '') # 系数后加空格，省略1
    else:
        coefficient = '%d/%d '%(n).as_integer_ratio() # 系数后加空格
    return coefficient

In [11]:
def to_formula(n): # 把化学反应计量数改写为化学反应方程式
    reactants = (-n[n<0]).apply(stoichiometric).to_frame().reset_index().iloc[:, ::-1].T.apply(''.join)
    products = n[n>0].apply(stoichiometric).to_frame().reset_index().iloc[:, ::-1].T.apply(''.join)
    formula = ' -> '.join(map(' + '.join, (reactants, products)))
    return formula

In [12]:
n = stoichiometry.drop(['enthalpy_experimental', 'enthalpy_calculated', 'OCO_backbone'], axis=1)
stoichiometry = stoichiometry[['enthalpy_experimental', 'enthalpy_calculated']]
stoichiometry = stoichiometry.assign(error_calculated=stoichiometry.enthalpy_experimental-stoichiometry.enthalpy_calculated)
stoichiometry = stoichiometry.assign(enthalpy_corrected=(n*(data.enthalpy_calculated+data.correction)).sum(axis=1))
stoichiometry = stoichiometry.assign(error_corrected=stoichiometry.enthalpy_experimental-stoichiometry.enthalpy_corrected)
stoichiometry = stoichiometry.set_index(n.T.apply(to_formula))

In [13]:
stoichiometry = stoichiometry.rename(columns={
    'enthalpy_experimental': 'ΔH_exp/eV', 'enthalpy_calculated': 'ΔH_calc/eV', 'error_calculated': 'Error_calc/eV',
    'enthalpy_corrected': 'ΔH_corr/eV', 'error_corrected': 'Error_corr/eV'
}).round(3)
stoichiometry.index.name = 'Stoichiometry'

In [14]:
stoichiometry.to_csv('OCO_backbone.csv')

### 导出数据
计算总反应自由能

In [15]:
data = data.assign(atoms_count=data.index.map(lambda s: Formula(s).count()))
relative_energy = data.free_energy + data.atoms_count.map(lambda count: get_relative_energy(get_balanced_count(count, pe=False), data.free_energy))
data = data.drop('atoms_count', axis=1).assign(relative_energy=relative_energy)
data.loc['N2', 'relative_energy'] = 0.
data.loc['NH3', 'relative_energy'] = data.free_energy['NH3'] - 0.5*data.free_energy['N2'] - 1.5*data.free_energy['H2']

In [16]:
data.to_csv('gas.csv')

#### SI
The calculated electronic energies (E_elec) were converted into free energies (G), by adding zero-point energies (ZPE), and enthalpic (H) and entropic (S) contributions of adsorbates obtained from a harmonic oscillator at T = 298.15 K. Free energy corrections for gaseous molecules were obtained from the ideal gas approximation at T = 298.15 K, see Supplementary Table S1.

Table S1. Free energy corrections for gaseous species at T = 298.15 K.

In [17]:
data.index.name = 'Species'
data = data.assign(TS=-data.entropy*data.temperature).round(3)
data.pressure = data.pressure.round().astype(int)
data = data[['potential_energy', 'ZPE_correction', 'pressure', 'TS', 'Cp', 'free_energy']]
data = data.rename(columns={
    'potential_energy': 'E_elec/eV', 'ZPE_correction': 'ZPE/eV',
    'pressure': 'Fugacity/Pa', 'TS': '-TS/eV',
    'Cp': '∫C_pdT/eV', 'free_energy': 'G/eV'})
data.to_csv('gas_SI.csv', encoding='utf_8_sig')